# Protocolo de Teletransporte na presença de ruídos
Para a implementação do protocolo, iniciaremos carregando as seguintes bibliotecas e funções.
Vale ressaltar que as bibliotecas nos fornecem os recursos necessários para o desenvolvimento das operações.
As funções **TensorProduct** e **sqrt** foram importadas para tornar as operações mais objetivas e a biblioteca **cmath** permite trabalhar com números complexos.

In [1]:
import numpy as np
import math
import sympy
from math import sqrt
import sys
import random
import cmath

### Qubits iniciais
Para iniciar o protocolo, iremos definir os possíveis estados de entrada dos qubits que serão operacionalizados.
As bases da Computação Quântica definem as bases $|0>$ e $|1>$ e sua representação matricial é descrita por $ \begin{bmatrix}
1 \\
0
\end{bmatrix}
$ e $ \begin{bmatrix}
0 \\
1
\end{bmatrix}
$ respectivamente.

### Operadores quânticos
Devem ser definidos também, todas os operadores/portas lógicas que atuarão durante o protocolo. Sendo eles os operadores ***CNOT***, ***Hadamard***, ***Pauli-X***, ***Pauli-Z*** e ***Identidade***.
Suas representações matriciais são descritas por:

<center>$ CNOT = \begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0
\end{bmatrix}
$ , $ H = \frac{1}{\sqrt2} \begin{bmatrix}
1 & 1 \\
1 & -1
\end{bmatrix}$ , $ X = \begin{bmatrix}
0 & 1 \\
1 & 0
\end{bmatrix}$, $ Z = \begin{bmatrix}
1 & 0 \\
0 & -1
\end{bmatrix}$ e $ I = \begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix}$.</center>


In [2]:
#Variáveis que descrevem os estados dos qubits
qbit0= np.matrix([1,0]).transpose()
qbit1= np.matrix([0,1]).transpose()

#Operadores quânticos
#CNOT
CNOT = np.matrix([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])

#Hadamard
H = 1/sqrt(2)*(np.matrix([[1,1], [1,-1]]))

#Pauli-X
X = np.matrix([[0, 1], [1, 0]])

#Pauli-Z
Z = np.matrix([[1, 0], [0, -1]])

#Identidade
I = np.matrix ([[1,0], [0,1]])

## Emaranhamento
O protocolo é iniciado com a realização do emaranhamento de modo que aplicaremos a porta ***H*** no estado $|0>$ e em seguida, definiremos o qubit alvo e o qubit controle para a aplicação da porta ***CNOT***. O qubit alvo e controle são definidos realizando o produto tensorial do primeiro pelo segundo de modo que:
$$ AC = \frac{1}{\sqrt2} \begin{bmatrix}
1 \\
1
\end{bmatrix} \otimes \begin{bmatrix}
1 \\
0
\end{bmatrix}
$$
Vale ressaltar que nesse caso estamos emaranhando dois qubits no estado $|0>$ criando uma das Bases de Bell que, nesse caso é descrita por:
$$\Phi_+ = \frac{1}{\sqrt2} \begin{bmatrix}
1 \\
0 \\
0 \\
1
\end{bmatrix}
$$

In [3]:
#Aplicação da porta Hadamard no qubit-0
Hqbit0= H*qbit0

#Definição Alvo e Controle
AC= np.kron(Hqbit0,qbit0)

#Aplicação da porta CNOT
Bell00 = CNOT * AC

Iremos agora definir o estado do qubit que será teletransportado. Para isso é solicitado ao usuário que insira as probabilidades de $\alpha$ e $\beta$ desejados. Lembrando que estas devem estar normalizadas e sua soma deve ser igual a 1, caso contrário o programa não pode continuar operando.
Como o estado do qubit generalizado é definido por:
$$ |\psi> = \alpha \begin{bmatrix}
1 \\
0
\end{bmatrix} + \beta \begin{bmatrix}
0 \\
1
\end{bmatrix}
$$
Atribuímos os valores $\alpha$ e $\beta$ ao estado inicial.

In [4]:
#Informação gerada para teletransporte

def gerar_funcao_psi():
    real_a = np.random.rand()  
    imag_a = np.random.rand()  
    real_b = np.random.rand()  
    imag_b = np.random.rand()  
    
    #Funções complexas a e b
    a = real_a + imag_a * 1j
    b = real_b + imag_b * 1j
    
    #Normalização para satisfazer |a|^2 + |b|^2 = 1
    norm = np.sqrt(abs(a)**2 + abs(b)**2)
    a /= norm
    b /= norm
    
    #Verificação da normalização
    condicao_normalizacao = np.isclose(abs(a)**2 + abs(b)**2, 1)
    
    #Crie a função psi = a|0⟩ + b|1⟩
    psi = a * np.array([1, 0]) + b * np.array([0, 1])
    
    return psi, a, b, condicao_normalizacao

psi, a, b, condicao_normalizacao = gerar_funcao_psi()

alfa_inicial=np.matrix([a,0]).transpose()
beta_inicial=np.matrix([0,b]).transpose()

print(alfa_inicial,beta_inicial)

[[0.5626207+0.55338843j]
 [0.       +0.j        ]] [[0.        +0.j        ]
 [0.08024828+0.60891659j]]


A próxima etapa do protocolo consiste na aplicação da porta ***CNOT*** nos três qubits que compõem o estado geral do sistema.
Os três qubits são o estado inicial a ser teletransportado atuando como controle e o par emaranhado atuando como alvo. A atuação ocorre apenas no qubit presente no local **A**, porém afeta probabilisticamente o qubit no local **B**.
Para definir o estado geral são somados os produtos tensoriais de todos os estados dos qubits. Em seguida, a porta ***CNOT*** é dimensionada para atuar sob três qubits com a realização do produto tensorial entre ***CNOT*** e ***I*** e por último multiplicamos a porta ***CNOT*** pela soma dos estados dos qubits. Esse estado é chamado de $|\psi_1>$ e pode ser definido por:
$$ |\psi_1> = \frac{1}{\sqrt2} \left\{ \alpha \begin{bmatrix}
1 \\
0
\end{bmatrix} \otimes  \begin{bmatrix}
1 \\
0 \\
0 \\
1
\end{bmatrix}  + \beta \begin{bmatrix}
0 \\
1
\end{bmatrix} \otimes  \begin{bmatrix}
0 \\
1 \\
1 \\
0
\end{bmatrix}  \right\}
$$
Em seguida, a porta ***H*** é aplicada no qubit a ser teletransportado. Sua aplicação foi realizada em cada um dos estados do qubit de maneira isolada para melhor observação da modificação dos estados deste. Após a aplicação da porta ***H*** o estado $|\psi_2>$ é definido por:
$$|\psi_2> = \frac{1}{2} \left\{ \alpha \begin{bmatrix}
1 \\
1
\end{bmatrix} \begin{bmatrix}
1 \\
0 \\
0 \\
1
\end{bmatrix} + \beta \begin{bmatrix}
1 \\
-1
\end{bmatrix} \begin{bmatrix}
0 \\
1 \\
1 \\
0
\end{bmatrix} \right\}
$$

In [5]:
#Estado geral (produto tensorial entre os estados de todos os qubits do sistema)
estado00 = np.kron(qbit0,qbit0)
estado01 = np.kron(qbit0, qbit1)
estado11 = np.kron(qbit1, qbit1)
estado10 = np.kron(qbit1,qbit0)
psi_0 = np.kron(estado00,qbit0) + np.kron(estado01, qbit1) + np.kron(estado10, qbit0) + np.kron(estado11,qbit1)

#Dimensionamento da porta CNOT para 3 qubits
CNOT_I = np.kron(CNOT,I)

#Aplicação da porta CNOT
psi_1 = CNOT_I * psi_0

#Aplicação da porta Hadamard
H_estado_alfa = H * alfa_inicial
H_estado_beta = H * beta_inicial

Para realizar a Medição e a Reconstrução do estado inicial, iremos separar os estados associados às probabilidades $\alpha$ e $\beta$ de modo que:
$$ \alpha \begin{bmatrix}
1 \\
1
\end{bmatrix} = \alpha \begin{bmatrix}
1 \\
0
\end{bmatrix} + \alpha \begin{bmatrix}
0 \\
1
\end{bmatrix} \quad e  \quad     \beta \begin{bmatrix}
1 \\
-1
\end{bmatrix} = \beta \begin{bmatrix}
1 \\
0 
\end{bmatrix} - \beta \begin{bmatrix}
0 \\
1
\end{bmatrix}
$$

In [6]:
#Separação dos estados Alfa e Beta
estadoa0 = H_estado_alfa[0][0]
estadoa1 = H_estado_alfa[1][0]
estadob0 = H_estado_beta[0][0]
estadob1 = H_estado_beta[1][0]

#Determinação dos estados do qbit enviado
a0 = np.array([complex(estadoa0[0]), 0])
a1 = np.array([0, complex(estadoa1[0])])
b0 = np.array([complex(estadob0[0]), 0])
b1 = np.array([0, complex(estadob1[0])])

Para realizar a medição, foram estabelecidos os possíveis estados presentes em **A**, e um deles foi escolhido de maneira aleatória, lembrando que a probabilidade de colapso em cada um dos estados é de $\frac{1}{4}$. Os estados que não foram escolhidos foram deletados, simulando o colapso do sistema.

In [7]:
#Medição
estado_00 = np.kron(qbit0, qbit0)
estado_11 = np.kron(qbit1, qbit1)
estado_10 = np.kron(qbit1, qbit0)
estado_01 = np.kron(qbit0, qbit1)

group_estados = [estado_00,estado_11,estado_10,estado_01]
chosen_estados = random.choice(group_estados)
Medição = chosen_estados

del estado_00, estado_11, estado_10, estado_01

print(Medição)

[[1]
 [0]
 [0]
 [0]]


As etapas a seguir acontecem no local **B** com o qubit do par emaranhado. Segundo o estado colapsado enviado por **A**, o qubit em **B** terá um estado correspondente de acordo com a relação:

Estado Medido em **A** | Estado do qubit emaranhado em **B**
:---------------------:|:----------------------------------:
$|00>$                 | $\alpha \begin{bmatrix} 1 \\ 0 \end{bmatrix} + \beta \begin{bmatrix} 0 \\ 1 \end{bmatrix}$
$|11>$                 | $\alpha \begin{bmatrix} 0 \\ 1 \end{bmatrix} - \beta \begin{bmatrix} 1 \\ 0 \end{bmatrix}$
$|01>$                 | $\alpha \begin{bmatrix} 0 \\ 1 \end{bmatrix} + \beta \begin{bmatrix} 1 \\ 0 \end{bmatrix}$
$|10>$                 | $\alpha \begin{bmatrix} 1 \\ 0 \end{bmatrix} - \beta \begin{bmatrix} 0 \\ 1 \end{bmatrix}$



In [8]:
#Definição do estado em B em função da medição realizada em A
if  np.all(Medição == (np.kron(qbit0,qbit0))):
    estado_tp = (a0-b1)
elif  np.all(Medição == (np.kron(qbit1,qbit1))):
    estado_tp = (a1-b0)
elif  np.all(Medição == (np.kron(qbit0,qbit1))):
    estado_tp = (a1+b0)
elif np.all(Medição == (np.kron(qbit1,qbit0))):
    estado_tp = (a0+b1)
    
estado_tp1=estado_tp.reshape(2,1)   
print(estado_tp1)

[[0.39783291+0.39130471j]
 [0.0567441 +0.43056905j]]


Agora, de acordo com o estado associado em B, uma sequência de operações deverão ser realizadas conforme a relação: 
Estado do qubit emaranhado em **B**                                                        | Operação realizada
:-----------------------------------------------------------------------------------------:|:------------------:
$\alpha \begin{bmatrix} 1 \\ 0 \end{bmatrix} + \beta \begin{bmatrix} 0 \\ 1 \end{bmatrix}$ | Nenhuma operação
$\alpha \begin{bmatrix} 0 \\ 1 \end{bmatrix} - \beta \begin{bmatrix} 1 \\ 0 \end{bmatrix}$ | X e Z
$\alpha \begin{bmatrix} 0 \\ 1 \end{bmatrix} + \beta \begin{bmatrix} 1 \\ 0 \end{bmatrix}$ | X
$\alpha \begin{bmatrix} 1 \\ 0 \end{bmatrix} - \beta \begin{bmatrix} 0 \\ 1 \end{bmatrix}$ | Z


In [9]:
if np.all(estado_tp == (a0-b1)):
    estado_final = estado_tp1
elif np.all(estado_tp == (a1-b0)):
    estado_final = Z * X * estado_tp1 
elif np.all(estado_tp == (a1+b0)):
    estado_final = X * estado_tp1 
elif np.all(estado_tp == (a0+b1)):
    estado_final = Z * estado_tp1 
    
estado_teletransportado = estado_final*sqrt(2)    

alfa = np.array(estado_teletransportado[0][0])
beta = np.array(estado_teletransportado[1][0])

alfa_final = np.matrix([complex(alfa),0]).transpose()
beta_final = np.matrix([0,complex(beta)]).transpose()

Se durante o processo, algum ruído ocorresse, o estado do qubit teletransportado seria alterado. A simulação de uma interação com os ruídos ***bitflip*** e ***phaseflip*** foi implementada de modo que o usuário pode selecionar se o teletransporte teve ou não ruído e em caso afirmativo, qual ruído ocorreu.
O ruído do tipo ***bitflip*** inverte o estado do qubit, ou seja, se este era $|0>$ passa a ser $|1>$ e vice-versa.
O ruído do tipo ***phaseflip*** inverte a fase do qubit.

In [10]:
#Aplicação de ruído
#Determinando os ruídos
bitflip = np.matrix([[0, 1], [1, 0]])
phaseflip = np.matrix([[1, 0], [0, -1]])
noisebit = 0

# Escolha do ruído aplicado
while True:
    noise = input('Escolha o ruído:[0]- sem ruído, [1]- bitflip, [2]- phaseflip:')
    if noise in ['0', '1', '2']:
        noise = int(noise)
        break
    else:
        print("Opção inválida. Por favor, digite 0, 1 ou 2.")

if noise == 1:
    noisebit = bitflip
elif noise == 2:
    noisebit = phaseflip
elif noise == 0:
    noisebit = 1

aplicação_noise_a = noisebit * alfa_final
aplicação_noise_b = noisebit * beta_final

alfa_noise = aplicação_noise_a
beta_noise = aplicação_noise_b

alfa_final = alfa_noise
beta_final = beta_noise

alfa_final = alfa_noise.reshape(2,1).astype(complex)
beta_final = beta_noise.reshape(2,1).astype(complex)

Info_TP = (alfa_final+beta_final)

Escolha o ruído:[0]- sem ruído, [1]- bitflip, [2]- phaseflip:1


Por fim, para verificar se o teletransporte funcionou e ainda, se houve ou não ruído e qual a natureza deste, a sequência de testes abaixo foi implementada.
Neles, comparamos os estados associados as probabilidades $\alpha$ e $\beta$ antes(inicial) e depois(final) do teletransporte.

In [11]:
if np.all((alfa_final == alfa_inicial) & (beta_final == beta_inicial)):
    print('Teletransporte concluído com sucesso e sem presença de ruídos', Info_TP)
else:
    if np.all((alfa_inicial[0][0] == alfa_final[1][0]) & (alfa_inicial[1][0] == alfa_final[0][0]) & (beta_inicial[1][0] == beta_final[0][0]) & (beta_inicial[0][0] == beta_final[1][0])):
        print("Mensagem teletransportada com ruído do tipo bitflip", Info_TP)
    elif np.all((alfa_inicial[0][0] == alfa_final[0][0]) & (alfa_inicial[1][0] == -alfa_final[1][0]) & (beta_inicial[1][0] == -beta_final[1][0]) &(beta_inicial[0][0] == beta_final[0][0])):
        print("Mensagem teletransportada com ruído do tipo phaseflip", Info_TP)
    else:
        print("Erro de teletransporte")

Mensagem teletransportada com ruído do tipo bitflip [[0.08024828+0.60891659j]
 [0.5626207 +0.55338843j]]


Vale ressaltar que o protocolo acima é uma simplificação de uma situação real e tem como finalidade compreender os procedimentos associados a ele.